# Imports

In [1]:
#Libraries
from WebScraperPageCount import WebScraper
import pandas as pd
import numpy as np
import os
import fnmatch
import tempfile
import psycopg2
from sqlalchemy import create_engine

os.environ['MPLCONFIGDIR'] = "/home/jovyan"
import matplotlib.pyplot as plt



## 1. Daten scrapen, Dataframe erstellen

In [ ]:
    dfImmo = WebScraper().runImmo()
    dfHome = WebScraper().runHome()



In [3]:
dfImmo.head(5)
   


,Size,Number of Rooms,Price,Street,Zip Code,City,State
0,108,4.5,2795,Geerenweg 1a,8157,Dielsdorf,Zurich
1,55,3.5,1360,Gotthelfstrasse 10,3400,Burgdorf,Bern
2,61,3.5,1450,Gotthelfstrasse 10,3400,Burgdorf,Bern
3,55,3.5,1390,Gotthelfstrasse 10,3400,Burgdorf,Bern
4,61,3.5,1510,Gotthelfstrasse 10,3400,Burgdorf,Bern


In [4]:
 dfHome.head(5)



,Size,Number of Rooms,Price,Street,Zip Code,City
0,84,3.5,3000,None,8952,Schlieren
1,65,3.5,1700,Gerichtshausstrasse 18,8750,Glarus
2,95,3.5,2490,Tüfistrasse 21,8107,Buchs ZH
3,100,4.5,1740,Via Pizzo di Claro 21,6500,Bellinzona
4,145,6,5500,None,1290,Versoix


In [5]:
data = pd.concat([dfImmo, dfHome])
data.head(5)




,Size,Number of Rooms,Price,Street,Zip Code,City,State
0,108,4.5,2795,Geerenweg 1a,8157,Dielsdorf,Zurich
1,55,3.5,1360,Gotthelfstrasse 10,3400,Burgdorf,Bern
2,61,3.5,1450,Gotthelfstrasse 10,3400,Burgdorf,Bern
3,55,3.5,1390,Gotthelfstrasse 10,3400,Burgdorf,Bern
4,61,3.5,1510,Gotthelfstrasse 10,3400,Burgdorf,Bern


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
   display(data)

In [7]:
data.to_excel('Output.xlsx', index=False)

## 2. Datenbereinigung

In [8]:
data.describe()

,Size,Number of Rooms,Price,Street,Zip Code,City,State
count,7998,7998,7998,7998,7998,7998,1008
unique,259,28,941,6078,1450,1528,25
top,None,3.5,1500,None,6900,Basel,Bern
freq,1109,2245,112,554,158,358,255


#### 2.1 Leere und falsche Adresseinträge auf NaN setzen

In [9]:
data['Street'].isnull().sum()
data_work = data
data_work['Street'].replace('', np.nan, inplace=True)
data_work['Street'].replace('None', np.nan, inplace=True)
data_work['Street'].replace('.', np.nan, inplace=True)
data_work['Street'].replace('44', np.nan, inplace=True)
data_work['Street'].replace('70', np.nan, inplace=True)
data_work['Price'].replace('None', np.nan, inplace=True)

In [10]:
data_work.describe()

,Size,Number of Rooms,Price,Street,Zip Code,City,State
count,7998,7998,7907,7426,7998,7998,1008
unique,259,28,940,6073,1450,1528,25
top,None,3.5,1500,Seestrasse 119/121,6900,Basel,Bern
freq,1109,2245,112,13,158,358,255


#### 2.2 Nan Einträge löschen

In [11]:
data_work.dropna(subset=['Street','Price'], inplace=True)

In [12]:
data_work.describe()

,Size,Number of Rooms,Price,Street,Zip Code,City,State
count,7343,7343,7343,7343,7343,7343,932
unique,240,23,916,6001,1333,1403,25
top,None,3.5,1650,Seestrasse 119/121,9000,Basel,Bern
freq,1026,2100,97,13,138,345,248


#### 2.3 Duplikate anhand von Adresse und Preis entfernen

In [13]:
data_cleansed = data_work.drop_duplicates(subset=['Price', 'Street'])
data_cleansed.describe()

,Size,Number of Rooms,Price,Street,Zip Code,City,State
count,6861,6861,6861,6861,6861,6861,921
unique,239,23,916,6001,1333,1397,25
top,None,3.5,1500,Höhenweg 18,9000,Basel,Bern
freq,989,1955,95,7,128,327,248


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
   display(data_cleansed)

## 3. Datenbank

#### 3.1 Verbindung zu Datenbank herstellen

In [45]:
conn = psycopg2.connect("host=db dbname=ads_project_db user=admin password=secret")

#### 3.2 Bereinigte Daten in Datenbank schreiben

In [46]:
engine = create_engine('postgresql://admin:secret@db:5432/ads_project_db')
data_cleansed.to_sql('immo_data', engine, if_exists='replace')

861

#### 3.3 SQL Abfrage

In [54]:
my_table    = pd.read_sql('''select 
                          "Size",
                          "Number of Rooms",
                          "Price",
                          "Street",
                          "Zip Code",
                          "City",
                          "State"                   
                          from immo_data''', conn)
print (my_table)

     Size Number of Rooms Price              Street Zip Code        City   
0     108             4.5  2795        Geerenweg 1a     8157   Dielsdorf  \
1      55             3.5  1360  Gotthelfstrasse 10     3400    Burgdorf   
2      61             3.5  1450  Gotthelfstrasse 10     3400    Burgdorf   
3      55             3.5  1390  Gotthelfstrasse 10     3400    Burgdorf   
4      61             3.5  1510  Gotthelfstrasse 10     3400    Burgdorf   
...   ...             ...   ...                 ...      ...         ...   
6856   63             3.5  1420        Vy d'Etra 35     2000   Neuchâtel   
6857   63             3.5  1380        Vy d'Etra 35     2000   Neuchâtel   
6858   63             3.5  1330        Vy d'Etra 35     2000   Neuchâtel   
6859  107             4.5  2090     Wiesenstrasse 4     5412  Gebenstorf   
6860   47               2  1190    Frobenstrasse 66     4053       Basel   

       State  
0     Zurich  
1       Bern  
2       Bern  
3       Bern  
4       Bern

/tmp/ipykernel_4936/3766181494.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  my_table    = pd.read_sql('''select
